In [1]:
import time
import datetime as dt
import sys, os
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
# CKM Packages
import dscore.ds.dshelpers as ds
# Process Mining
fuzzy_mapper_loc = '/Users/ksha/Documents/ospm/modules/fuzzy_mapper_class/'
process_mining_loc = '/Users/ksha/Documents/pm-core/src/'
package_locations = [fuzzy_mapper_loc, process_mining_loc]
for loc in package_locations:
    sys.path.append(loc)
import mapper
from process_mining_core import ping_pong, routing
from utils.ebi_parser import parsed_data

/Users/ksha/anaconda3/envs/ckm_base/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# UIHC Modeling

__Simulation Wishlist:__
- Incorporate holidays
- Set capacity limits for non-OR/ER nodes.
- Have ping-pong effects (PAT back to origin, post-op back to OR, post-op back to pre-op, long-term care back to scheduled)
- Schedule micro-delays in the OR after patient moves from the OR to POST-OP nodes to represent OR clean-up
- [OPTIONAL] Have patients that arrive at the queue first, get treated first (SEE NOTES).

__Simulation ReadMe:__

- Modelling Patient Arrival Times
    - The interval between patient arrivals is estimated based on a user input for the number of patients processed per year. 
        - _Running Example: UIHC's Main OR for FY2019 = 19490. This indicates a rate of 1 patient every 0.45 hours._
    - Patient types are broken down into SCHEDULED, FLOOR, or ED TRAUMA by frequency of occurence.
        - _Let's say UIHC is 50% SCHEDULED, 30% FLOOR, and 20% ED TRAUMA._
    - Non-emergency (SCHEDULED and FLOOR) patients are assumed to __arrive__ during business hours (9AM-5PM). There are 2,088 business hours in 2019. This is used to calculate the expected hour intervals between arrivals.
        - $$\frac{2088}{0.8 \times 19490} = 0.13$$
    - Emergency (ED TRAUMA) patients arrive on a 24/7 basis.
        - $$\frac{365 \times 24}{0.2 \times 19490} = 2.25$$
    - The actual arrival intervals are then randomly sampled from the exponential distribution where the scale parameter is set to the expected intervals calculated above.

- Modelling Pre-processing Stages
    - Unlike the ORs, there's no capacity limit for PAT, REGISTERED, PRE-OP, and QUEUE nodes.
    - Due to randomization, patients will travel through these nodes at different rates, so overtaking is possible.
     
- Modelling OR and ER Queues and Operations
     - Once they've arrived at the queue, patients are treated __strictly__ in the order of their arrival at the hospital.
         - _E.g. There are 12 ORs. Patient $i$ is the $i^{th}$ patient to arrive. Regardless of how long they take to move through the pre-processing stages, patients 1-12 will __not have to queue__ for ORs 1-12. Let's assume patient 13 moves quickly through pre-processing, arriving at the OR QUEUE __before__ patients 1-12. She __still has to queue__, allowing patients 1-12 overtake her into the ORs. Patient 13 takes the next OR that opens up._
     - There is at most 1 patient per OR / ER at any one time.
     - If there's a queue, the end time of an operation is the start time of the next patient's operation. No lag time accounting for hospital turnover is included.
- Modelling Post-processing Stages
    - Unlike the ORs, there's no capacity limit for POST-OP nodes.
    - The patient's mode of discharge (DISCHARGED, LONG-TERM CARE, or FLOOR), is selected from a matrix of probabilities depending on whether they were SCHEDULED, FLOOR, or ED TRAUMA.

# Reformatting

## Breakdown of Surgical Specialties
Operating times are loosely defined from the following link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5609617/pdf/1679-4508-eins-15-02-0200.pdf

In [2]:
specialty_dict = {'specialty': ['Anesthesia', 'Dentistry', 'Dermatology', 'Gynecology', 'Int Med - BMT',
                                'Neurology', 'Neurosurgery', 'Ophthalmology', 'Orthopaedics', 'Otolaryngology',
                                'Radiology', 'Surgery', 'Urology'],
                  'operating_time': [2.969, 2, 3, 2.141, 2,
                                     0.6, 3.64, 2.867, 3.8, 3.248,
                                     2, 3.63, 2.567
                                     ],
                  'frequency': [0.0009, 0.0316, 0.0007, 0.0679, 0.0008,
                                0.0145, 0.0914, 0.1182, 0.252, 0.1075,
                                0.0042, 0.2515, 0.0588]
                  }
df_specialty = pd.DataFrame(specialty_dict).set_index('specialty')
df_specialty

,operating_time,frequency
specialty,,
Anesthesia,2.969,0.0009
Dentistry,2.000,0.0316
Dermatology,3.000,0.0007
Gynecology,2.141,0.0679
Int Med - BMT,2.000,0.0008
Neurology,0.600,0.0145
Neurosurgery,3.640,0.0914
Ophthalmology,2.867,0.1182
Orthopaedics,3.800,0.2520


In [3]:
def specialty_generator(df):
    """
    Function to generate the medical field that the patient's operation is categorized in,
    and the corresponding average operating time, based on an input dataframe (df_specialty).
    """
    sp_type = np.random.choice(a=df_specialty.index,
                               p=df_specialty.frequency)
    or_duration = df.loc[sp_type].operating_time
    return sp_type, or_duration

## Breakdown of Appointment Type and Outcome 

In [4]:
pt_dict = {'appointment': ['SCHEDULED', 'ED TRAUMA', 'FLOOR'],
           'frequency': [0.5, 0.2, 0.3],
           'DISCHARGED': [0.5, 0.1, 0.2],
           'LONG-TERM CARE': [0.4, 0.3, 0.6],
           'FLOOR': [0.1, 0.6, 0.2]
           }
df_pt = pd.DataFrame(pt_dict).set_index('appointment')
df_pt

,frequency,DISCHARGED,LONG-TERM CARE,FLOOR
appointment,,,,
SCHEDULED,0.5,0.5,0.4,0.1
ED TRAUMA,0.2,0.1,0.3,0.6
FLOOR,0.3,0.2,0.6,0.2


In [5]:
def pt_generator(df):
    """
    Function to generate a patient's manner of appointment for the operation,
    and the mode of their discharge, based on an input dataframe (df_pt).
    """
    pt_type = np.random.choice(a=df.index, p=df.frequency)
    dc_type = np.random.choice(a=df.loc[pt_type][1:].index, p=df.loc[pt_type][1:])
    return pt_type, dc_type

## Timestamp Generator

In [6]:
def ts_generator(time_list):
    """
    Function to generate sequential timestamps based on cumulatively summing samples from an exponential
    distribution, based on an input list of expected average wait times between events.
    """
    deltas = [time_list[0]]
    for i in time_list[1:]:
        gen = np.random.exponential(i)
        deltas.append(gen)
    return list(np.cumsum(deltas))

## Building Patient Class

In [7]:
class Patient:
    """
    Given inputs, constructs a profile of a patient that is arrives at a hospital. 
    
    Attributes
    ----------
    pt_number : int 
        Patient number - a count that increments everytime a patient profile is instantiated.
        This also marks the order in which patient arrivals occur.
    id : int
        ID unique to each `Patient` instance.
    pt_type : str
        The patient's appointment type (SCHEDULED, ED TRAUMA, or from hospital FLOORs)
    dc_type : str
        How the patient is discharged (DISCHARGED, LONG-TERM CARE, or back to hospital FLOORs)
    sp_type : str
        The medical specialty that the surgery falls under.
    or_time : float
        The expected time taken to complete surgery given the sp_type.
    ed_start_ts : float
        Denotes the arrival time of patients who need the emergency room.
        Only present for ED TRAUMA patients, who can arrive at any time of the day. 
    start_ts : float
        Denotes the arrival time of non-emergency patient cases. Only present for 
        SCHEDULED and FLOOR patients, who only arrive during business hours (9AM-5PM).
    """
    pt_number = 0
    start_ts = pd.Timestamp(dt.datetime(2019, 1, 1)).timestamp()
    ed_start_ts = pd.Timestamp(dt.datetime(2019, 1, 1)).timestamp()

    def __init__(self, pt_df, specialty_df, num_pt_per_year=19490):
        """
        Parameters
        ----------
        pt_df : Pandas dataframe
            Matrix of probabilities denoting the likelihood of occurrence of each patient type,
            and their corresponding mode of discharge.
        specialty_df : Pandas dataframe
            Dataframe describing the likelihood that a surgery relates to a medical specialty,
            and the corresponding expected surgery time in that specialty.
        num_pt_per_year : int
            Number of patients that the practice processes every year, defaults to 19,490.
        """
        # Increment patient counter and assign to instance attribute
        Patient.pt_number += 1
        self.pt_number = Patient.pt_number

        # Generate unique patient ID
        self.id = id(self)

        # Generate where patient arrives from, and mode of discharge
        self.pt_type, self.dc_type = pt_generator(pt_df)

        # Generate medical specialty, and corresponding average operating time
        self.sp_type, self.or_time = specialty_generator(specialty_df)

        # Estimate average time interval between patient arrivals
        freq_series = pt_df.frequency * num_pt_per_year
        ed_trauma_interval = (365*24) / \
            freq_series['ED TRAUMA']  # ER operates 24/7
        scheduled_floor_interval = 2088 / \
            freq_series[['SCHEDULED', 'FLOOR']].sum()  # Business hours only

        # Generate start time for patient arrival drawn from exponential distribution
        if self.pt_type == 'ED TRAUMA':
            Patient.ed_start_ts += np.random.exponential(dt.timedelta(hours=ed_trauma_interval)
                                                         .total_seconds())
            self.ed_start_ts = Patient.ed_start_ts
        else:
            Patient.start_ts += np.random.exponential(dt.timedelta(hours=scheduled_floor_interval)
                                                      .total_seconds())
            # Use an offset to adjust timestamp to valid business hours if needed
            offset = pd.offsets.BusinessHour()
            Patient.start_ts = offset.rollforward(
                pd.Timestamp.utcfromtimestamp(Patient.start_ts)).timestamp()
            self.start_ts = Patient.start_ts

## Building Simulation Class

In [8]:
class Simulation:
    """
    Given a `Patient` instance, generates an event log of that patient's path through
    the hospital floor.
    
    Attributes
    ----------
    pt : `Patient` instance object 
        An instance of the `Patient` class representing a patient profile.
    or_arrival_time : float
        Denotes the arrival time of SCHEDULED and FLOOR patients to the OR QUEUE. With each new instance,
        corresponding class attribute is updated with the latest arrival.
    er_arrival_time : float
        Denotes the arrival time of ED TRAUMA patients to the ER QUEUE. With each new instance,
        corresponding class attribute is updated with the latest arrival.
    hosp_dict : dict
        Dictionary to populate the patient's event log with.
    """
    or_arrival_time = {'OR 1': 0.0, 'OR 2': 0.0, 'OR 3': 0.0, 'OR 4': 0.0,
                       'OR 5': 0.0, 'OR 6': 0.0, 'OR 7': 0.0, 'OR 8': 0.0,
                       'OR 9': 0.0, 'OR 10': 0.0, 'OR 11': 0.0, 'OR 12': 0.0, }
    er_arrival_time = {'ER 1': 0.0, 'ER 2': 0.0, 'ER 3': 0.0}

    def __init__(self, pt_instance):
        """
        Parameters
        ----------
        pt_instance : `Patient` instance object
            An instance of the `Patient` class representing a patient profile.
        """
        self.pt = pt_instance
        self.or_arrival_time = Simulation.or_arrival_time
        self.er_arrival_time = Simulation.er_arrival_time
        self.hosp_dict = {'ticket': [], 'patient_id': [],
                          'node': [], 'specialty': [], 'ts': []}

    def generate_data(self):
        """
        Please see help(Simulation) for more info
        """
        # Outpatient surgery scheduled ahead of time, or inpatient surgery from 'floors'
        if (self.pt.pt_type == 'SCHEDULED') or (self.pt.pt_type == 'FLOOR'):

            # Increment the patient count
            self.hosp_dict['ticket'].extend(8*[self.pt.pt_number])

            # Extend Patient ID
            self.hosp_dict['patient_id'].extend(8*[self.pt.id])

            # Have patients go to the next available OR
            min_wait = min(self.or_arrival_time, key=self.or_arrival_time.get)

            # Extend nodes with patient's path through hospital floor
            self.hosp_dict['node'].extend([self.pt.pt_type, 'PAT', 'REGISTERED', 'PRE-OP', 'OR QUEUE',
                                           min_wait, 'POST-OP', self.pt.dc_type
                                           ])
            # Extend medical specialty type
            self.hosp_dict['specialty'].extend(8*[self.pt.sp_type])

            # Generate and extend timestamps until queueing stage
            self.intervals_1 = ts_generator(time_list=[self.pt.start_ts,
                                                       dt.timedelta(
                                                           hours=1).total_seconds(),
                                                       dt.timedelta(
                                                           hours=1).total_seconds(),
                                                       dt.timedelta(
                                                           hours=0.5).total_seconds(),
                                                       dt.timedelta(
                                                           hours=1).total_seconds()
                                                       ])
            # Check whether queueing is required extend timestamps until end
            if self.intervals_1[-1] >= self.or_arrival_time[min_wait]:
                self.intervals_2 = ts_generator(time_list=[self.intervals_1[-1],
                                                           dt.timedelta(
                                                               hours=self.pt.or_time).total_seconds(),
                                                           dt.timedelta(
                                                               hours=1.5).total_seconds()
                                                           ])
                Simulation.or_arrival_time[min_wait] = self.intervals_2[1]
            else:
                self.intervals_2 = ts_generator(time_list=[self.or_arrival_time[min_wait],
                                                           dt.timedelta(
                                                               hours=self.pt.or_time).total_seconds(),
                                                           dt.timedelta(
                                                               hours=1.5).total_seconds()
                                                           ])
                Simulation.or_arrival_time[min_wait] = self.intervals_2[1]
            self.hosp_dict['ts'].extend(self.intervals_1 + self.intervals_2)

        # Unscheduled patient requires emergency treatment
        elif self.pt.pt_type == 'ED TRAUMA':
            self.hosp_dict['ticket'].extend(5*[self.pt.pt_number])
            self.hosp_dict['patient_id'].extend(5*[self.pt.id])
            min_wait = min(self.er_arrival_time, key=self.er_arrival_time.get)
            self.hosp_dict['node'].extend(
                ['ED TRAUMA', 'ER QUEUE', min_wait, 'POST-OP', self.pt.dc_type])
            self.hosp_dict['specialty'].extend(5*[self.pt.sp_type])
            self.intervals_1 = ts_generator(time_list=[self.pt.ed_start_ts,
                                                       dt.timedelta(
                                                           hours=0.25).total_seconds()
                                                       ])
            if self.intervals_1[-1] >= self.er_arrival_time[min_wait]:
                self.intervals_2 = ts_generator(time_list=[self.intervals_1[-1],
                                                           dt.timedelta(
                                                               hours=1.5*self.pt.or_time).total_seconds(),
                                                           dt.timedelta(
                                                               hours=2.5).total_seconds()
                                                           ])
                Simulation.er_arrival_time[min_wait] = self.intervals_2[1]
            else:
                self.intervals_2 = ts_generator(time_list=[self.er_arrival_time[min_wait],
                                                           dt.timedelta(
                                                               hours=1.5*self.pt.or_time).total_seconds(),
                                                           dt.timedelta(
                                                               hours=2.5).total_seconds()
                                                           ])
                Simulation.er_arrival_time[min_wait] = self.intervals_2[1]
            self.hosp_dict['ts'].extend(self.intervals_1 + self.intervals_2)

# Simulation 

In [9]:
start = time.time()

df_sim = {'ticket': [], 'patient_id': [],
          'node': [], 'specialty': [], 'ts': []}
for i in range(1000):

    # Call Patient class
    pt = Patient(num_pt_per_year=19490, pt_df=df_pt, specialty_df=df_specialty)
    sim = Simulation(pt)
    sim.generate_data()

    # Append to simulation dictionary
    df_sim['ticket'].extend(sim.hosp_dict['ticket'])
    df_sim['patient_id'].extend(sim.hosp_dict['patient_id'])
    df_sim['node'].extend(sim.hosp_dict['node'])
    df_sim['specialty'].extend(sim.hosp_dict['specialty'])
    df_sim['ts'].extend(sim.hosp_dict['ts'])

end = time.time()
print(end-start, 'runtime')

3.660813093185425 runtime


In [10]:
df_sim = pd.DataFrame(data=df_sim)
df_sim.patient_id = df_sim.patient_id.astype(int)
df_sim.ticket = df_sim.ticket.astype(int)

In [11]:
df_sim.analyze_cols()

,percent_null,distinct_count,data_type,top_value,top_share
ticket,0,1000,int64,999,0.00108196
patient_id,0,165,int64,4808702216,0.071815
node,0,26,object,POST-OP,0.135245
specialty,0,13,object,Surgery,0.248174
ts,0,6394,float64,1.54645e+09,0.00027049


In [12]:
df_clean = df_sim.copy()
df_clean.ts = df_sim.ts.apply(lambda x: pd.Timestamp.utcfromtimestamp(x))
df_clean.head()

,ticket,patient_id,node,specialty,ts
0,1,4819507128,SCHEDULED,Dentistry,2019-01-01 09:00:00.000000
1,1,4819507128,PAT,Dentistry,2019-01-01 09:28:29.975690
2,1,4819507128,REGISTERED,Dentistry,2019-01-01 10:06:45.272460
3,1,4819507128,PRE-OP,Dentistry,2019-01-01 10:19:48.306508
4,1,4819507128,OR QUEUE,Dentistry,2019-01-01 12:19:19.543907


# EDA

In [13]:
df_sort = df_clean.sort_values(by='ts', kind='mergesort').reset_index(drop=True).drop(columns='ticket')
df_sort.to_csv('hospital_sim.csv')
df_sort.head()

,patient_id,node,specialty,ts
0,4819507408,ED TRAUMA,Orthopaedics,2019-01-01 01:25:13.103563
1,4819507408,ER QUEUE,Orthopaedics,2019-01-01 01:27:07.419317
2,4819507408,ER 1,Orthopaedics,2019-01-01 01:27:07.419317
3,4819507408,POST-OP,Orthopaedics,2019-01-01 04:07:34.973739
4,4819507408,FLOOR,Orthopaedics,2019-01-01 04:14:17.454221


In [14]:
df_sort['node_delta'] = df_sort.groupby('node').ts.diff().dt.total_seconds()/60
df_sort.groupby('node').node_delta.describe()

,count,mean,std,min,25%,50%,75%,max
node,,,,,,,,
DISCHARGED,332.0,87.684016,215.353812,0.118946,18.145905,44.403798,88.269671,2261.988700
ED TRAUMA,201.0,137.252339,132.039551,0.838700,45.245306,103.870913,180.732845,716.552398
ER 1,59.0,472.320206,363.846029,3.948673,218.678488,363.528995,673.368968,1523.737755
ER 2,64.0,427.967734,416.928651,14.619902,129.710692,316.004145,619.079404,2226.678993
ER 3,76.0,360.170387,260.941285,23.284515,140.411055,294.208125,547.348171,1117.252929
ER QUEUE,201.0,137.268866,134.542572,1.259213,42.505929,106.002481,182.848961,705.976810
FLOOR,543.0,55.645343,97.936226,0.118851,8.490634,22.150140,56.883989,744.060302
LONG-TERM CARE,438.0,67.126823,131.268134,0.104371,12.472672,32.178307,67.050556,1506.538822
OR 1,66.0,439.653401,758.633006,3.205281,102.692912,183.719922,439.470794,3860.601470


# Process Mapping New Simulation

In [15]:
mp = mapper.Mapper(event_log=df_sim)
mp.population()

PRE-OP 0.798
OR 5 0.061
ER 1 0.06
ER QUEUE 0.202
OR 12 0.072
SCHEDULED 0.482
OR 10 0.068
OR 11 0.067
OR 7 0.068
OR 8 0.067
ED TRAUMA 0.202
OR 6 0.072
OR 4 0.073
REGISTERED 0.798
ER 2 0.065
ER 3 0.077
OR 1 0.067
LONG-TERM CARE 0.439
DISCHARGED 0.333
OR 2 0.049
PAT 0.798
OR 3 0.063
OR QUEUE 0.798
OR 9 0.071
POST-OP 1.0
FLOOR 0.544


In [16]:
mp.fit(add_st_end=False, preserve_threshold=0.25, ratio_threshold=0.1,
       utility_ratio=0.75, edge_cutoff=0.1, node_cutoff=0, ranks=True)

Number of conforming tickets:  1000
Number of nonconforming tickets:  0
{4: ['<PRE-OP>'], 6: ['<OR 5>', '<OR 12>', '<OR 10>', '<OR 11>', '<OR 7>', '<OR 8>', '<OR 6>', '<OR 4>', '<OR 1>', '<OR 2>', '<OR 3>', '<OR 9>'], 3: ['<ER 1>', '<REGISTERED>', '<ER 2>', '<ER 3>'], 2: ['<ER QUEUE>', '<PAT>'], 1: ['<SCHEDULED>', '<ED TRAUMA>', '<FLOOR>'], 8: ['<LONG-TERM CARE>', '<DISCHARGED>'], 5: ['<OR QUEUE>'], 7: ['<POST-OP>']}


In [17]:
mp.generate_html(filename='hospital_sim.html',sample=1000, include_s_e=True)

z-score ok tickets:  958 out of  1000
mean_ttime is 38934.40912063734
